# Attribution par facteur
D’où vient le rendement d'un fonds ?  
On explique les rendements excédentaires du fonds par les facteurs académiques :
* βi​ = sensibilité du fonds au facteur i
* α = performance non expliquée par ces facteurs

Données utilisées :
* NAV mensuelles ou hebdomadaires du fonds/ETF
* Facteurs Fama-French

Description du fichier de facteurs :

`MonthId`
* Type : entier (YYYYMM)
* Description : Identifiant du mois de l’observation - Exemple : 196307 = juillet 1963.
* Usage : Sert d’index temporel pour aligner facteurs, rendements du fonds et séries macro.

`Mkt-RF`
* Nom complet : Excess Market Return
* Type : float (%)
* Définition : Rendement du portefeuille de marché (value-weighted US equities) moins le taux sans risque (RF).
* Interprétation : Facteur de risque de marché (le premier facteur CAPM).
* Usage en attribution : Charge en bêta du fonds → exposition directionnelle au marché.

`SMB`
* Nom complet : Small Minus Big
* Type : float (%)
* Définition : Surperformance moyenne des portefeuilles "petites capitalisations" sur les grandes capitalisations.
* Construction 2×3 : Moyenne de 6 portefeuilles classés par taille × 3 niveaux de “book-to-market”, “operating profitability”, “investment”.
* Interprétation : Prime de taille.
* Usage en attribution : Exposition aux small caps (portefeuilles spécialisés, value/small tilt, etc.).

`HML`
* Nom complet : High Minus Low (Value Factor)
* Type : float (%)
* Définition : Surperformance moyenne des actions Value (haut book-to-market) vs Growth (bas book-to-market).
* Construction 2×3 : Portefeuilles triés taille × book-to-market.
* Interprétation : Prime Value.
* Usage : Explique les biais Value/Growth.

`RMW`
* Nom complet : Robust Minus Weak (Profitability Factor)
* Type : float (%)
* Définition : Surperformance des entreprises à profitabilité robuste vs faible.
* Construction 2×3 : Portefeuilles triés taille × operating profitability.
* Interprétation : Prime de profitabilité (Cooper, Gulen, Schill ; Fama-French 2015).
* Usage : Détecte un tilt vers les entreprises “quality/profitable”.

`CMA`
* Nom complet : Conservative Minus Aggressive (Investment Factor)
* Type : float (%)
* Définition : Surperformance des entreprises faiblement investisseuses vs fortement investisseuses.
* Interprétation : Facteur d’investissement — entreprises prudentes vs expansion agressive.
* Usage : Explique les biais vers les entreprises conservatrices / capital-light.

`RF`
* Nom complet : Risk-Free Rate
* Type : float (%)
* Définition : Taux sans risque à un mois (T-bill 1-month).
* Usage : Permet de calculer les rendements excédentaires du fonds et sert au facteur Mkt-RF.

In [1]:
import pandas as pd
import statsmodels.api as sm

In [2]:
# Charger FF5
df = pd.read_csv('FF5+Momentum.csv')

In [3]:
# Encodage date
df['month_enddate'] = pd.to_datetime(df['MonthId'], format='%Y%m') + pd.offsets.MonthEnd(0)
df = df.set_index('month_enddate').drop(columns=['MonthId'])

In [4]:
# Charger rendements du fonds (alignés mensuellement)
fund = pd.read_csv('VLUE.csv')
fund['month_enddate'] = pd.to_datetime(
    fund['month_enddate'],
    format='%d-%b-%y'
) + pd.offsets.MonthEnd(0)

fund = fund.set_index('month_enddate')

In [5]:
# Alignement
merged = fund.join(df, how='inner')

In [6]:
# Rendement excédentaire du fonds
merged['fund_excess'] = merged['monthly_total_return'] - merged['RF']

In [8]:
# Statsmodels OLS
X = merged[['Mkt-RF', 'SMB', 'HML', 'RMW', 'CMA','UMD']]
X = sm.add_constant(X)
y = merged['fund_excess']

model = sm.OLS(y, X).fit(cov_type='HAC', cov_kwds={'maxlags':6})
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:            fund_excess   R-squared:                       0.934
Model:                            OLS   Adj. R-squared:                  0.931
Method:                 Least Squares   F-statistic:                     355.7
Date:                Sat, 15 Nov 2025   Prob (F-statistic):           6.59e-83
Time:                        10:26:31   Log-Likelihood:                -242.85
No. Observations:                 149   AIC:                             499.7
Df Residuals:                     142   BIC:                             520.7
Df Model:                           6                                         
Covariance Type:                  HAC                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.1686      0.088     -1.922      0.0